<a href="https://colab.research.google.com/github/greensky0107/self_study/blob/main/Day29_Classification_Decision_Tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Decision Tree

지도학습 알고리즘 (분류, 회귀)

직관적인 알고리즘 (이해 쉬움)

과대적합되기 쉬운 알고리즘 (트리 깊이 제한 필요)

트리의 루트(root)에서 시작해서 정보이득이 최대가 되는 특성으로 데이터를 나눔.

정보이득(information gain)이 최대가 되는 특성을 나누는 기준(불순도를 측정하는 기준)은 '지니'와 '엔트로피'가 사용됨.

데이터가 한 종류만 있다면 엔트로피/지니 불순도는 0에 가깝고, 서로 다른 데이터의 비율이 비슷하면 1에 가까움.

정보이득(information gain)이 최대라는 것은 불순도를 최소화 하는 방향(1-불순도)

*   Gini계수 사용 시 :  불순도를 최소화하는 방향으로 데이터를 나누어 나감.

*   엔트로피 사용 시 :  정보이득을 최대화하는 방향으로 데이터를 나누어 나감.


In [21]:
#Step 1. Library 불러오기

# Pandas 불러오기
import pandas as pd

# Numpy 불러오기
import numpy as np

# Matplotlib 불러오기
import matplotlib.pyplot as plt

# Seaborn 불러오기
import seaborn as sns

# 사이킷런 내장데이터 확인
import sklearn.datasets
sklearn.datasets.__all__  # load로 된 부분이 데이터셋

# 데이터 불러오기
from sklearn.datasets import load_breast_cancer  # load_데이터 세트

# 데이터 분리
from sklearn.model_selection import train_test_split

# 모델 불러오기
# 의사결정나무
from sklearn.tree import DecisionTreeClassifier
# 랜덤포레스트
from sklearn.ensemble import RandomForestClassifier
# XGBoost
from xgboost import XGBClassifier

# 평가
# accuracy (정확도)
from sklearn.metrics import accuracy_score
# precision (정밀도)
from sklearn.metrics import precision_score
# recall (재현율)
from sklearn.metrics import recall_score
# f1 (정밀도와 재현율의 조화평균)
from sklearn.metrics import f1_score
# roc auc (Receiver Operation Chracteristic Curve,  Area Under the Curve)
from sklearn.metrics import roc_auc_score

In [23]:
#Step2 Data Split (train vs. test.  size 얼마로 할 지 정하기.  50%, 30%, 20%, 10%)

from sklearn.datasets import load_breast_cancer

def make_dataset():
    cancer = load_breast_cancer()
    df = pd.DataFrame(cancer.data, columns=cancer.feature_names)
    df['target'] = cancer.target
    X_train, X_test, y_train, y_test = train_test_split(
        df.drop('target', axis=1), df['target'], test_size=0.5, random_state=5000)
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = make_dataset()
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((284, 30), (285, 30), (284,), (285,))

In [24]:
#Step3 Target 확인
y_train.value_counts()

,count
target,
1,170
0,114


In [26]:
#Step 4 Decision Tree 실행

# 모델 변수 정의_ dt_clf(의사결정나무) / rf_clf(랜덤포레스트) / lr_clf(로지스틱회귀) 등
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(random_state=0)

#학습
model.fit(X_train, y_train)

#예측
pred = model.predict(X_test)

#평가
# 정확도
accuracy_score(y_test, pred)

#test size 50% 일때 0.922    #test size 30% 일때 0.935

0.9228070175438596

Decision Tree Hyperparameter

criterion (기본값 gini) : 불순도 지표 (또는 엔트로피 불순도 entropy)

max_depth (기본값 None) : 최대 한도 깊이

min_samples_split (기본값 2) : 자식 노드를 갖기 위한 최소한의 데이터 수

min_samples_leaf (기본값 1) : 리프 노드가 되기 위한 최소 샘플 수

In [27]:
#Step 5 Tune Decision Tree Hyperparameter (max depth, minimum sample leaf, minimum sample split 조정)

from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(
    criterion = 'entropy',
    max_depth = 7,
    min_samples_split = 2,
    min_samples_leaf=2,
    random_state=0)
model.fit(X_train, y_train)
pred = model.predict(X_test)

# 정확도
accuracy_score(y_test, pred)

#test size 50%, max_depth=7, min_samples_split=2, min_sampls_leaf=2 일 때, 0.943
#test size 30%, max_depth=7, min_samples_split=2, min_sampls_leaf=2 일 때, 0.888
#test size 30%, max_depth=10, min_samples_split=5, min_sampls_leaf=5 일 때, 0.923
#test size 30%, max_depth=10, min_samples_split=10, min_sampls_leaf=10 일 때, 0.941
#test size 30%, max_depth=15, min_samples_split=15, min_sampls_leaf=15 일 때, 0.923

0.9438596491228071

In [30]:
#Step 5 Tune Decision Tree Hyperparameter (max depth, minimum sample leaf, minimum sample split 조정)

# 정밀도
precision_score(y_test, pred)

0.9523809523809523

In [31]:
#Step 5 Tune Decision Tree Hyperparameter (max depth, minimum sample leaf, minimum sample split 조정)

# 재현율
recall_score(y_test, pred)

0.9625668449197861

In [32]:
#Step 5 Tune Decision Tree Hyperparameter (max depth, minimum sample leaf, minimum sample split 조정)

# f1
f1_score(y_test, pred)

0.9574468085106383

In [34]:
#Step 5 Tune Decision Tree Hyperparameter (max depth, minimum sample leaf, minimum sample split 조정)

# roc_auc
model = XGBClassifier(random_state=0, eval_metric='logloss')
model.fit(X_train, y_train)
pred = model.predict_proba(X_test)
roc_auc_score(y_test, pred[:,1])

np.float64(0.9855669540543491)

[Decision Tree 예제]

UCI Human Activitiy Recongnition Using Smartphones Dataset  (2012년 data)

In [9]:
#Google Drive Data 가져오기

from google.colab import drive
drive.mount('/content/drive') # Changed the mount point to '/content/drive'

feature_name_df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/UCI_HAR_Dataset/human_activity/features.txt',sep='\s+',
                            header=None,names=['column_index','column_name'])

feature_name=feature_name_df.iloc[:,1].values.tolist()
print('전체 피처명에서 10개만 추출:',feature_name[:10])

MessageError: Error: credential propagation was unsuccessful

In [ ]:
feature_dup_df=feature_name_df.groupby('column_name').count()
print(feature_dup_df[feature_dup_df['column_index']>1].count())
feature_dup_df[feature_dup_df['column_index']>1].head()

column_index    42
dtype: int64


,column_index
column_name,
"fBodyAcc-bandsEnergy()-1,16",3
"fBodyAcc-bandsEnergy()-1,24",3
"fBodyAcc-bandsEnergy()-1,8",3
"fBodyAcc-bandsEnergy()-17,24",3
"fBodyAcc-bandsEnergy()-17,32",3


In [ ]:
def get_new_feature_name_df(old_feature_name_df):
    feature_dup_df=pd.DataFrame(data=old_feature_name_df.groupby('column_name').cumcount(),columns=['dup_cnt'])
    feature_dup_df=feature_dup_df.reset_index()
    new_feature_name_df=pd.merge(old_feature_name_df.reset_index(),feature_dup_df,how='outer')
    new_feature_name_df['column_name']=new_feature_name_df[['column_name','dup_cnt']].apply(lambda x:x[0]+'_'+str(x[1]) if x[1]>0 else x[0], axis=1)
    new_feature_name_df=new_feature_name_df.drop(['index'],axis=1)
    return new_feature_name_df

In [ ]:
import pandas as pd

def feature_name_df(feature_name_df):
  new_feature_name_df=get_new_feature_name_df(feature_name_df)
  feature_name=new_feature_name_df.ser.iloc[pos].values.tolist()
  X_train=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/UCI_HAR_Dataset/human_activity/train/X_train.txt',sep='\s+', names=feature_name)
  X_test=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/UCI_HAR_Dataset/human_activity/test/X_test.txt',sep='\s+', names=feature_name)
  y_train=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/UCI_HAR_Dataset/human_activity/train/y_train.txt',sep='\s+', names=['y_train'])
  y_test=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/UCI_HAR_Dataset/human_activity/test/y_test.txt',sep='\s+', names=['y_test'])
  return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test=get_human_dataset()
print(X_train.info())

<ipython-input-60-31e13317fa9f>:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  new_feature_name_df['column_name']=new_feature_name_df[['column_name','dup_cnt']].apply(lambda x:x[0]+'_'+str(x[1]) if x[1]>0 else x[0], axis=1)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7352 entries, 0 to 7351
Columns: 561 entries, tBodyAcc-mean()-X to angle(Z,gravityMean)
dtypes: float64(561)
memory usage: 31.5 MB
None


In [ ]:
dt_clf=DecisionTreeClassifier(random_state=156)
dt_clf.fit(X_train, y_train)
pred=dt_clf.predict(X_test)
accuracy=accuracy_score(y_test, pred)
print('결정 트리 예측 정확도: {0:.4f}'.format(accuracy))

print('DecisionTreeClassifier 기본 하이퍼 파라미터:\n',dt_clf.get_params())


ValueError: Input y contains NaN.